# Run PCR-GLOBWB Sophie

In [11]:
from grpc4bmi.bmi_client_singularity import BmiClientSingularity
from pathos.threading import ThreadPool as Pool

from pathlib import Path
from cftime import num2date

import glob
import pandas as pd
import numpy as np
import xarray as xr

## Load Functions

In [12]:
def var_to_xarray(model, variable):
    # Get grid properties from model (x = latitude !!)
    # could be speedup, lots of bmi calls are done here that dont change between updates
    shape = model.get_grid_shape(model.get_var_grid(variable))
    lat = model.get_grid_x(model.get_var_grid(variable))
    lon = model.get_grid_y(model.get_var_grid(variable))
    time = num2date(model.get_current_time(), model.get_time_units())

    # Get model data for variable at current timestep
    data = model.get_value(variable)
    data = np.reshape(data, shape)

    # Create xarray object
    da = xr.DataArray(data, 
                      coords = {'longitude': lon, 'latitude': lat, 'time': time}, 
                      dims = ['latitude', 'longitude'],
                      name = variable,
                      attrs = {'units': model.get_var_units(variable)}
                     )

    # Masked invalid values on return array:
    return da.where(da != -999)


def bmi_model_run(singularity_image, input_folder, output_folder, config_file):
    
    # Create output list
    model_output = []
    
    
    # Create model object (singularity image)
    print('Create model object')
    model = BmiClientSingularity(image=singularity_image,
                                 input_dir=input_folder,
                                 output_dir=output_folder)
    
    # Initialize model object
    print('Initialize model object')
    model.initialize(config_file)
    
    # Get time units
    time_units = model.get_time_units()
    
    print(f'Running Model Run', flush=True)

    # Run model between specified start and end in the configfile
    while model.get_current_time() < model.get_end_time(): 
        #Get the current time to print
        time = num2date(model.get_current_time(), time_units)
        print(f'Current time: {time}', end="\r")

        # Run the model for the next timestep
        model.update()
        
        # Append output
        model_output.append(var_to_xarray(model, variable="total_evaporation"))
    
    # Finalize model
    model.finalize()
    
    # Write output netcdf
    ds_out = xr.concat(model_output, dim='time')
    ds_out.to_netcdf(f'{output_folder}/{output_folder.split("/")[-2]}_total_evaporation.nc')
    
    return model_output


def parallel_model_run(singularity_images, input_folders, output_folders, config_files, threads=None):
    # Set number of threads (cores) used for parallel run and map threads
    if threads is None:
        pool = Pool()
    else:
        pool = Pool(nodes=threads)

    pool.map(
        bmi_model_run,
        singularity_images,
        input_folders,
        output_folders,
        config_files,
    )
    
    return print('parallel_finished')

## Set Paths and Files

In [13]:
WORKDIR = Path('/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/')
INPUTDIR = Path('/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/')
OUTPUTDIR = Path('/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/')

# flux_selection_file = 'fluxnet_toren_selectie_klein_Sophie_Jaren.csv'
flux_selection_file = 'rerun.csv'
singularity_image = "/home/jaerts/ewatercycle-pcrg-grpc4bmi.sif"

## Load Dataframe

In [23]:
df = pd.read_csv(f'{WORKDIR}/{flux_selection_file}', sep=';')
df = df.set_index('SITE_ID')
df

,SITE_NAME,FLUXNET2015,FLUXNET-CH4,LOCATION_LAT,LOCATION_LONG,LOCATION_ELEV,IGBP,MAT,MAP,Aantal jaar,Klimaat,Vanaf,Tot,totaal
SITE_ID,,,,,,,,,,,,,,
AU-Cpr,Calperum,CC-BY-4.0,NaN,-34.0021,140.5891,NaN,SAV,NaN,NaN,5,BWk,2011-01,2013-12,3 jaar


## Create lists for parallel run

In [24]:
singularity_images = []
input_folders = []
output_folders = []
config_files = []

for index, row in df.iterrows():
    

    singularity_images.append(singularity_image)
    input_folders.append(str(INPUTDIR))

    output_folder = f'{OUTPUTDIR}/{index.lower()}_output/'
    output_folders.append(output_folder)
    
    config_file = glob.glob(f'{INPUTDIR}/{index.lower()}_05min_Input.ini')[0]
    config_files.append(config_file)

## Start Parallel Run

In [25]:
len(input_folders)

1

In [26]:
parallel_model_run(singularity_images, input_folders, output_folders, config_files)

Create model object
Initialize model object
Running Model Run
parallel_finished3-12-30 00:00:00
